In [1]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from difflib import SequenceMatcher

# ================= FILE PATHS =================
OLD_FILE = "Old_ones.xlsx"
NEW_FILE = "New_one.xlsx"
OUTPUT_FILE = "Fixture_Comparison_Report.xlsx"

# ================= LOAD FILES =================
old_df = pd.read_excel(OLD_FILE)
new_df = pd.read_excel(NEW_FILE)

cols = ["Start Date", "Start Time", "Description", "Venue"]
old_df = old_df[cols].copy()
new_df = new_df[cols].copy()

# ================= NORMALIZE DESCRIPTION =================
def normalize_desc(text):
    if pd.isna(text):
        return ""
    return " ".join(text.lower().split())

old_df["Desc_norm"] = old_df["Description"].apply(normalize_desc)
new_df["Desc_norm"] = new_df["Description"].apply(normalize_desc)

# ================= FUZZY MATCH FUNCTION =================
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

MATCH_THRESHOLD = 0.9  # 90% similarity

# ================= PAIR FUZZY MATCHES =================
matched_new = set()
pairs = []

for i, old_row in old_df.iterrows():
    best_match = None
    best_score = 0

    for j, new_row in new_df.iterrows():
        if j in matched_new:
            continue

        score = similarity(old_row["Desc_norm"], new_row["Desc_norm"])
        if score > best_score:
            best_score = score
            best_match = j

    if best_score >= MATCH_THRESHOLD:
        matched_new.add(best_match)
        pairs.append((i, best_match))
    else:
        pairs.append((i, None))

# Add unmatched NEW rows
for j in new_df.index:
    if j not in matched_new:
        pairs.append((None, j))

# ================= BUILD COMPARISON =================
rows = []

for old_i, new_i in pairs:
    old = old_df.loc[old_i] if old_i is not None else None
    new = new_df.loc[new_i] if new_i is not None else None

    if old is None:
        change_type = "ADDED"
    elif new is None:
        change_type = "REMOVED"
    else:
        changes = []

        if old["Description"] != new["Description"]:
            changes.append("Description")
        if old["Start Date"] != new["Start Date"]:
            changes.append("Start Date")
        if old["Start Time"] != new["Start Time"]:
            changes.append("Start Time")
        if old["Venue"] != new["Venue"]:
            changes.append("Venue")

        if changes:
            change_type = "MODIFIED (" + ", ".join(changes) + ")"
        else:
            change_type = "NO CHANGE"

    rows.append({
        "Change Type": change_type,
        "Description_OLD": old["Description"] if old is not None else "",
        "Description_NEW": new["Description"] if new is not None else "",
        "Start Date_OLD": old["Start Date"] if old is not None else "",
        "Start Time_OLD": old["Start Time"] if old is not None else "",
        "Venue_OLD": old["Venue"] if old is not None else "",
        "Start Date_NEW": new["Start Date"] if new is not None else "",
        "Start Time_NEW": new["Start Time"] if new is not None else "",
        "Venue_NEW": new["Venue"] if new is not None else "",
    })

final_report = pd.DataFrame(rows)

# ================= EXPORT =================
final_report.to_excel(OUTPUT_FILE, index=False)

# ================= APPLY HIGHLIGHTING =================
wb = load_workbook(OUTPUT_FILE)
ws = wb.active

FILL_ADDED = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
FILL_REMOVED = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
FILL_MODIFIED = PatternFill(start_color="FFEB9C", end_color="FFEB9C", fill_type="solid")

header = [cell.value for cell in ws[1]]
change_col_idx = header.index("Change Type") + 1

for row in range(2, ws.max_row + 1):
    val = ws.cell(row=row, column=change_col_idx).value

    if val == "ADDED":
        fill = FILL_ADDED
    elif val == "REMOVED":
        fill = FILL_REMOVED
    elif val.startswith("MODIFIED"):
        fill = FILL_MODIFIED
    else:
        continue

    for col in range(1, ws.max_column + 1):
        ws.cell(row=row, column=col).fill = fill

wb.save(OUTPUT_FILE)

print(f"✅ Fixture comparison report with spelling detection created: {OUTPUT_FILE}")


✅ Fixture comparison report with spelling detection created: Fixture_Comparison_Report.xlsx
